# Yolo txt to PASCAL VOC xml Format

In [1]:
import os
import xml.etree.ElementTree as ET

In [ ]:
import os
import xml.etree.ElementTree as ET

def yolo_to_voc(yolo_file, img_width, img_height, class_list, output_dir):
    with open(yolo_file, 'r') as file:
        yolo_lines = file.readlines()

    filename = os.path.basename(yolo_file).replace('.txt', '')

    annotation = ET.Element("annotation")
    ET.SubElement(annotation, "folder").text = "images"
    ET.SubElement(annotation, "filename").text = f"{filename}.jpg"
    
    size = ET.SubElement(annotation, "size")
    ET.SubElement(size, "width").text = str(img_width)
    ET.SubElement(size, "height").text = str(img_height)
    ET.SubElement(size, "depth").text = "3"
    
    for line in yolo_lines:
        yolo_data = line.strip().split()

        try:
            class_id = int(yolo_data[0])
            x_center, y_center, bbox_width, bbox_height = map(float, yolo_data[1:])
        except (ValueError, IndexError):
            print(f"Skipping malformed line in {yolo_file}: {line.strip()}")
            continue

        x_min = int((x_center - bbox_width / 2) * img_width)
        y_min = int((y_center - bbox_height / 2) * img_height)
        x_max = int((x_center + bbox_width / 2) * img_width)
        y_max = int((y_center + bbox_height / 2) * img_height)

        obj = ET.SubElement(annotation, "object")
        ET.SubElement(obj, "name").text = class_list[class_id]
        ET.SubElement(obj, "pose").text = "Unspecified"
        ET.SubElement(obj, "truncated").text = "0"
        ET.SubElement(obj, "difficult").text = "0"

        bndbox = ET.SubElement(obj, "bndbox")
        ET.SubElement(bndbox, "xmin").text = str(x_min)
        ET.SubElement(bndbox, "ymin").text = str(y_min)
        ET.SubElement(bndbox, "xmax").text = str(x_max)
        ET.SubElement(bndbox, "ymax").text = str(y_max)

    tree = ET.ElementTree(annotation)
    output_file = os.path.join(output_dir, f"{filename}.xml")
    os.makedirs(output_dir, exist_ok=True)
    tree.write(output_file, encoding="utf-8", xml_declaration=True)

    print(f"Converted {yolo_file} to {output_file}")

def batch_convert_yolo_to_voc(yolo_dir, img_width, img_height, class_list, output_dir):
    yolo_files = [f for f in os.listdir(yolo_dir) if f.endswith('.txt')]

    for yolo_file in yolo_files:
        yolo_path = os.path.join(yolo_dir, yolo_file)
        yolo_to_voc(yolo_path, img_width, img_height, class_list, output_dir)

if __name__ == "__main__":
    yolo_dir = r"Line Detection using Yolo On PalmLeaf\Label"
    img_width = 640
    img_height = 640
    class_list = ["line"]
    output_dir = "output_voc"
    batch_convert_yolo_to_voc(yolo_dir, img_width, img_height, class_list, output_dir)

Converted Line Detection using Yolo On PalmLeaf\Label\001_nl_b334_051_01.txt to output_voc\001_nl_b334_051_01.xml
Converted Line Detection using Yolo On PalmLeaf\Label\002_nl_b393_077_01.txt to output_voc\002_nl_b393_077_01.xml
Converted Line Detection using Yolo On PalmLeaf\Label\003_sr_bo_05_001.txt to output_voc\003_sr_bo_05_001.xml
Converted Line Detection using Yolo On PalmLeaf\Label\004_sr_bo_06_001.txt to output_voc\004_sr_bo_06_001.xml
Converted Line Detection using Yolo On PalmLeaf\Label\005_nl_a128_173_02.txt to output_voc\005_nl_a128_173_02.xml
Converted Line Detection using Yolo On PalmLeaf\Label\006_sr_bo_05_004.txt to output_voc\006_sr_bo_05_004.xml
Converted Line Detection using Yolo On PalmLeaf\Label\007_efeo_201_01_04.txt to output_voc\007_efeo_201_01_04.xml
Converted Line Detection using Yolo On PalmLeaf\Label\008_nl_a161_304_01.txt to output_voc\008_nl_a161_304_01.xml
Converted Line Detection using Yolo On PalmLeaf\Label\009_sr_bo_07_004.txt to output_voc\009_sr_bo_0

In [3]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import pandas as pd
import xml.etree.ElementTree as ET

def voc_to_csv(voc_dir, output_csv):
    data = []

    for xml_file in os.listdir(voc_dir):
        if xml_file.endswith('.xml'):
            xml_path = os.path.join(voc_dir, xml_file)
            tree = ET.parse(xml_path)
            root = tree.getroot()
            filename = root.find('filename').text

            for obj in root.findall('object'):
                class_name = obj.find('name').text
                bndbox = obj.find('bndbox')
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)
                data.append([filename, class_name, xmin, ymin, xmax, ymax])
    
    df = pd.DataFrame(data, columns=['filename', 'class', 'xmin', 'ymin', 'xmax', 'ymax'])
    df.to_csv(output_csv, index=False)

if __name__ == "__main__":
    voc_dir = "output_voc"
    output_csv = r"Line Detection YoloNAS\annotations.csv"  
    voc_to_csv(voc_dir, output_csv)

In [12]:
data_ground_truth = pd.read_csv(output_csv)
data_ground_truth

,filename,class,xmin,ymin,xmax,ymax
0,001_nl_b334_051_01.jpg,line,40,50,594,129
1,001_nl_b334_051_01.jpg,line,40,150,594,228
2,001_nl_b334_051_01.jpg,line,44,274,598,352
3,001_nl_b334_051_01.jpg,line,44,387,598,465
4,001_nl_b334_051_01.jpg,line,44,498,598,577
...,...,...,...,...,...,...
3292,717_nl_a143_025_01.jpg,line,69,46,574,155
3293,717_nl_a143_025_01.jpg,line,69,155,574,263
3294,717_nl_a143_025_01.jpg,line,69,249,574,357
3295,717_nl_a143_025_01.jpg,line,69,380,574,488


In [13]:
if __name__ == "__main__":
    voc_dir = r"Line Detection YoloNAS\Prediction_K"
    output_csv = r"Line Detection YoloNAS\predicted_annotations1.csv"
    voc_to_csv(voc_dir, output_csv)

In [14]:
data_predicted = pd.read_csv(output_csv)
data_predicted

,filename,class,xmin,ymin,xmax,ymax
0,001_nl_b334_051_01_jpg.rf.b97331df70fc6f3c188b...,line,42,52,595,130
1,001_nl_b334_051_01_jpg.rf.b97331df70fc6f3c188b...,line,42,151,595,230
2,001_nl_b334_051_01_jpg.rf.b97331df70fc6f3c188b...,line,45,275,599,354
3,001_nl_b334_051_01_jpg.rf.b97331df70fc6f3c188b...,line,46,388,599,467
4,001_nl_b334_051_01_jpg.rf.b97331df70fc6f3c188b...,line,46,500,599,578
...,...,...,...,...,...,...
3278,717_nl_a143_025_01_jpg.rf.39c62aee2c11bb7f17c5...,line,71,48,576,156
3279,717_nl_a143_025_01_jpg.rf.39c62aee2c11bb7f17c5...,line,70,156,575,265
3280,717_nl_a143_025_01_jpg.rf.39c62aee2c11bb7f17c5...,line,71,250,576,359
3281,717_nl_a143_025_01_jpg.rf.39c62aee2c11bb7f17c5...,line,71,381,576,490


In [19]:
%pip install tqdm

# Performance Matrices

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
from tqdm import tqdm

def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area_box1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area_box1 + area_box2 - intersection
    return intersection / union if union > 0 else 0

def calculate_hausdorff(box1, box2):
    points1 = np.array([[box1[0], box1[1]], [box1[2], box1[1]], [box1[2], box1[3]], [box1[0], box1[3]]])
    points2 = np.array([[box2[0], box2[1]], [box2[2], box2[1]], [box2[2], box2[3]], [box2[0], box2[3]]])
    distances = cdist(points1, points2)
    return max(distances.min(axis=1).max(), distances.min(axis=0).max())

def calculate_avg_hd(box1, box2):
    points1 = np.array([[box1[0], box1[1]], [box1[2], box1[1]], [box1[2], box1[3]], [box1[0], box1[3]]])
    points2 = np.array([[box2[0], box2[1]], [box2[2], box2[1]], [box2[2], box2[3]], [box2[0], box2[3]]])
    distances = cdist(points1, points2)
    return (distances.min(axis=1).mean() + distances.min(axis=0).mean()) / 2

def calculate_hd95(box1, box2):
    points1 = np.array([[box1[0], box1[1]], [box1[2], box1[1]], [box1[2], box1[3]], [box1[0], box1[3]]])
    points2 = np.array([[box2[0], box2[1]], [box2[2], box2[1]], [box2[2], box2[3]], [box2[0], box2[3]]])
    distances = cdist(points1, points2).flatten()
    return np.percentile(distances, 95)

def evaluate_metrics(predicted, ground_truth):
    results = []
    for _, gt_row in tqdm(ground_truth.iterrows(), total=len(ground_truth)):
        gt_filename = gt_row['filename']
        gt_box = [gt_row['xmin'], gt_row['ymin'], gt_row['xmax'], gt_row['ymax']]
        pred_rows = predicted[predicted['filename'] == gt_filename]
        if pred_rows.empty:
            continue
        best_iou, best_hd, best_avg_hd, best_hd95 = 0, float('inf'), float('inf'), float('inf')
        for _, pred_row in pred_rows.iterrows():
            pred_box = [pred_row['xmin'], pred_row['ymin'], pred_row['xmax'], pred_row['ymax']]
            iou = calculate_iou(gt_box, pred_box)
            hd = calculate_hausdorff(gt_box, pred_box)
            avg_hd = calculate_avg_hd(gt_box, pred_box)
            hd95 = calculate_hd95(gt_box, pred_box)
            best_iou = max(best_iou, iou)
            best_hd = min(best_hd, hd)
            best_avg_hd = min(best_avg_hd, avg_hd)
            best_hd95 = min(best_hd95, hd95)
        results.append([gt_filename, best_iou, best_hd, best_avg_hd, best_hd95])
    return pd.DataFrame(results, columns=['filename', 'IoU', 'HD', 'AvgHD', 'HD95'])

if __name__ == "__main__":
    predicted = pd.read_csv(r'Line Detection YoloNAS\predicted_annotations1.csv')
    ground_truth = pd.read_csv(r'Line Detection YoloNAS\annotations.csv')
    predicted['filename'] = predicted['filename'].str.split('_jpg.rf').str[0] + '.jpg'
    metrics_df = evaluate_metrics(predicted, ground_truth)
    print(metrics_df)
    overall_metrics = metrics_df[['IoU', 'HD', 'AvgHD', 'HD95']].mean()
    print("Overall Metrics:")
    print(overall_metrics)
    metrics_df.to_csv('per_file_metrics.csv', index=False)
    overall_metrics.to_csv('overall_metrics.csv', header=True)

100%|██████████| 3297/3297 [00:09<00:00, 359.32it/s]

                    filename       IoU        HD     AvgHD        HD95
0     001_nl_b334_051_01.jpg  0.957384  2.828427  2.178694  560.421018
1     001_nl_b334_051_01.jpg  0.957405  2.828427  2.178694  560.421018
2     001_nl_b334_051_01.jpg  0.959096  2.236068  1.825141  560.421018
3     001_nl_b334_051_01.jpg  0.957405  2.828427  2.178694  560.421018
4     001_nl_b334_051_01.jpg  0.957384  2.828427  2.178694  560.421018
...                      ...       ...       ...       ...         ...
3278  717_nl_a143_025_01.jpg  0.965157  2.828427  2.532248  518.324875
3279  717_nl_a143_025_01.jpg  0.968935  2.236068  1.825141  517.346766
3280  717_nl_a143_025_01.jpg  0.965157  2.828427  2.532248  518.324875
3281  717_nl_a143_025_01.jpg  0.965157  2.828427  2.532248  518.324875
3282  717_nl_a143_025_01.jpg  0.967047  2.828427  2.178694  517.346766

[3283 rows x 5 columns]
Overall Metrics:
IoU        0.958306
HD         2.670132
AvgHD      2.350376
HD95     530.307075
dtype: float64


# .tif to .jpg

In [ ]:
import os
from PIL import Image

def convert_tif_to_jpg(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    tif_files = [f for f in os.listdir(input_dir) if f.lower().endswith('.tif')]
    for idx, tif_file in enumerate(tif_files, start=1):
        input_path = os.path.join(input_dir, tif_file)
        output_path = os.path.join(output_dir, f"{idx}.jpg")
        try:
            with Image.open(input_path) as img:
                img = img.convert("RGB")
                img.save(output_path, "JPEG")
            print(f"Converted {tif_file} to {output_path}")
        except Exception as e:
            print(f"Error converting {tif_file}: {e}")

input_directory = "Line Detection using Yolo On PalmLeaf\Sundanese\Images"
output_directory = "Line Detection using Yolo On PalmLeaf\Sundanese\Output_Image"
convert_tif_to_jpg(input_directory, output_directory)

<>:34: SyntaxWarning: invalid escape sequence '\S'
<>:35: SyntaxWarning: invalid escape sequence '\S'
<>:34: SyntaxWarning: invalid escape sequence '\S'
<>:35: SyntaxWarning: invalid escape sequence '\S'
C:\Users\7b148\AppData\Local\Temp\ipykernel_2196\2733339030.py:34: SyntaxWarning: invalid escape sequence '\S'
  input_directory = "Line Detection using Yolo On PalmLeaf\Sundanese\Images"  # Replace with the path to your .tif files
C:\Users\7b148\AppData\Local\Temp\ipykernel_2196\2733339030.py:35: SyntaxWarning: invalid escape sequence '\S'
  output_directory = "Line Detection using Yolo On PalmLeaf\Sundanese\Output_Image"  # Replace with the desired output directory


Converted CB-3-18-90-1.tif to Line Detection using Yolo On PalmLeaf\Sundanese\Output_Image\1.jpg
Converted CB-3-18-90-10.tif to Line Detection using Yolo On PalmLeaf\Sundanese\Output_Image\2.jpg
Converted CB-3-18-90-12.tif to Line Detection using Yolo On PalmLeaf\Sundanese\Output_Image\3.jpg
Converted CB-3-18-90-3.tif to Line Detection using Yolo On PalmLeaf\Sundanese\Output_Image\4.jpg
Converted CB-3-18-90-4.tif to Line Detection using Yolo On PalmLeaf\Sundanese\Output_Image\5.jpg
Converted CB-3-18-90-5.tif to Line Detection using Yolo On PalmLeaf\Sundanese\Output_Image\6.jpg
Converted CB-3-18-90-6.tif to Line Detection using Yolo On PalmLeaf\Sundanese\Output_Image\7.jpg
Converted CB-3-18-90-7.tif to Line Detection using Yolo On PalmLeaf\Sundanese\Output_Image\8.jpg
Converted CB-3-18-90-8.tif to Line Detection using Yolo On PalmLeaf\Sundanese\Output_Image\9.jpg
Converted CB-3-18-90-9.tif to Line Detection using Yolo On PalmLeaf\Sundanese\Output_Image\10.jpg
Converted CB-3-19-90-1.tif 

# Dataset 2

In [ ]:
import os
import xml.etree.ElementTree as ET

def yolo_to_voc(yolo_file, img_width, img_height, class_list, output_dir):
    with open(yolo_file, 'r') as file:
        yolo_lines = file.readlines()

    filename = os.path.basename(yolo_file).replace('.txt', '')

    annotation = ET.Element("annotation")
    ET.SubElement(annotation, "folder").text = "images"
    ET.SubElement(annotation, "filename").text = f"{filename}.jpg"
    
    size = ET.SubElement(annotation, "size")
    ET.SubElement(size, "width").text = str(img_width)
    ET.SubElement(size, "height").text = str(img_height)
    ET.SubElement(size, "depth").text = "3"
    
    for line in yolo_lines:
        yolo_data = line.strip().split()
        try:
            class_id = int(yolo_data[0])
            x_center, y_center, bbox_width, bbox_height = map(float, yolo_data[1:])
        except (ValueError, IndexError):
            continue

        x_min = int((x_center - bbox_width / 2) * img_width)
        y_min = int((y_center - bbox_height / 2) * img_height)
        x_max = int((x_center + bbox_width / 2) * img_width)
        y_max = int((y_center + bbox_height / 2) * img_height)

        obj = ET.SubElement(annotation, "object")
        ET.SubElement(obj, "name").text = class_list[class_id]
        ET.SubElement(obj, "pose").text = "Unspecified"
        ET.SubElement(obj, "truncated").text = "0"
        ET.SubElement(obj, "difficult").text = "0"

        bndbox = ET.SubElement(obj, "bndbox")
        ET.SubElement(bndbox, "xmin").text = str(x_min)
        ET.SubElement(bndbox, "ymin").text = str(y_min)
        ET.SubElement(bndbox, "xmax").text = str(x_max)
        ET.SubElement(bndbox, "ymax").text = str(y_max)

    tree = ET.ElementTree(annotation)
    output_file = os.path.join(output_dir, f"{filename}.xml")
    os.makedirs(output_dir, exist_ok=True)
    tree.write(output_file, encoding="utf-8", xml_declaration=True)

def batch_convert_yolo_to_voc(yolo_dir, img_width, img_height, class_list, output_dir):
    yolo_files = [f for f in os.listdir(yolo_dir) if f.endswith('.txt')]
    for yolo_file in yolo_files:
        yolo_path = os.path.join(yolo_dir, yolo_file)
        yolo_to_voc(yolo_path, img_width, img_height, class_list, output_dir)

if __name__ == "__main__":
    yolo_dir = r"Line Detection using Yolo On PalmLeaf\Sundanese\Label"
    img_width = 640
    img_height = 640
    class_list = ["line"]
    output_dir = "output_sun_voc"
    batch_convert_yolo_to_voc(yolo_dir, img_width, img_height, class_list, output_dir)

Converted Line Detection using Yolo On PalmLeaf\Sundanese\Label\1.txt to output_sun_voc\1.xml
Converted Line Detection using Yolo On PalmLeaf\Sundanese\Label\10.txt to output_sun_voc\10.xml
Converted Line Detection using Yolo On PalmLeaf\Sundanese\Label\11.txt to output_sun_voc\11.xml
Converted Line Detection using Yolo On PalmLeaf\Sundanese\Label\12.txt to output_sun_voc\12.xml
Converted Line Detection using Yolo On PalmLeaf\Sundanese\Label\13.txt to output_sun_voc\13.xml
Converted Line Detection using Yolo On PalmLeaf\Sundanese\Label\14.txt to output_sun_voc\14.xml
Converted Line Detection using Yolo On PalmLeaf\Sundanese\Label\15.txt to output_sun_voc\15.xml
Converted Line Detection using Yolo On PalmLeaf\Sundanese\Label\16.txt to output_sun_voc\16.xml
Converted Line Detection using Yolo On PalmLeaf\Sundanese\Label\17.txt to output_sun_voc\17.xml
Converted Line Detection using Yolo On PalmLeaf\Sundanese\Label\18.txt to output_sun_voc\18.xml
Converted Line Detection using Yolo On Pal

In [18]:
if __name__ == "__main__":
    voc_dir = "output_sun_voc"
    output_csv = r"Line Detection YoloNAS\annotations2.csv"
    voc_to_csv(voc_dir, output_csv)

In [19]:
data_ground_truth2 = pd.read_csv(output_csv)
data_ground_truth2

,filename,class,xmin,ymin,xmax,ymax
0,1.jpg,line,19,31,629,145
1,1.jpg,line,16,198,626,312
2,1.jpg,line,13,341,626,443
3,1.jpg,line,10,473,623,575
4,10.jpg,line,14,43,623,161
...,...,...,...,...,...,...
363,93.jpg,line,48,185,452,290
364,93.jpg,line,44,320,448,426
365,93.jpg,line,50,435,564,564
366,95.jpg,line,119,90,520,228


In [21]:
if __name__ == "__main__":
    voc_dir = r"Line Detection YoloNAS\Prediction_S"
    output_csv = r"Line Detection YoloNAS\predicted_annotations2.csv"
    voc_to_csv(voc_dir, output_csv)

In [13]:
data_predicted2 = pd.read_csv('predicted_annotations2.csv')
data_predicted2

,filename,class,xmin,ymin,xmax,ymax
0,10_jpg.rf.7298cdbf699a4309d552c2e80e8c3afc.jpg,line,16,44,625,163
1,10_jpg.rf.7298cdbf699a4309d552c2e80e8c3afc.jpg,line,15,174,624,293
2,10_jpg.rf.7298cdbf699a4309d552c2e80e8c3afc.jpg,line,15,325,624,443
3,10_jpg.rf.7298cdbf699a4309d552c2e80e8c3afc.jpg,line,14,464,623,583
4,11_jpg.rf.8fdc0b66c07960d107feb569839bb9e2.jpg,line,21,83,613,204
...,...,...,...,...,...,...
237,8_jpg.rf.1d51fcd61c7317bf22233e92dfa48cce.jpg,line,4,485,632,590
238,9_jpg.rf.90c07de89b62539627c04882903e7496.jpg,line,10,35,633,165
239,9_jpg.rf.90c07de89b62539627c04882903e7496.jpg,line,11,187,636,310
240,9_jpg.rf.90c07de89b62539627c04882903e7496.jpg,line,14,310,637,443


In [ ]:
if __name__ == "__main__":
    predicted = pd.read_csv(r'Line Detection YoloNAS\predicted_annotations2.csv')
    ground_truth = pd.read_csv(r'Line Detection YoloNAS\annotations2.csv')

    predicted['filename'] = predicted['filename'].str.split('_jpg.rf').str[0] + '.jpg'

    metrics_df = evaluate_metrics(predicted, ground_truth)
    
    print(metrics_df)
    overall_metrics = metrics_df[['IoU', 'HD', 'AvgHD', 'HD95']].mean()
    print("Overall Metrics:")
    print(overall_metrics)

    metrics_df.to_csv('per_file_metrics.csv', index=False)
    overall_metrics.to_csv('overall_metrics.csv', header=True)

100%|██████████| 368/368 [00:00<00:00, 540.49it/s]

    filename       IoU        HD     AvgHD        HD95
0      1.jpg  0.960858  2.828427  2.532248  621.363741
1      1.jpg  0.979420  1.414214  1.414214  621.453116
2      1.jpg  0.955404  2.828427  2.828427  623.240660
3      1.jpg  0.974266  2.236068  2.236068  623.320126
4     10.jpg  0.968696  2.828427  2.532248  622.244075
..       ...       ...       ...       ...         ...
237    8.jpg  0.956618  2.828427  2.828427  638.528714
238    9.jpg  0.963584  2.828427  2.828427  638.176210
239    9.jpg  0.969656  2.828427  2.532248  638.712524
240    9.jpg  0.971589  2.828427  2.532248  638.943798
241    9.jpg  0.969804  2.828427  2.178694  614.992430

[242 rows x 5 columns]
Overall Metrics:
IoU        0.966697
HD         2.667580
AvgHD      2.351992
HD95     617.855301
dtype: float64


In [ ]:
if __name__ == "__main__":
    yolo_dir = r"Line Detection using Yolo On PalmLeaf\Balinese\Label"
    img_width = 640
    img_height = 640
    class_list = ["line"]
    output_dir = "output_bali_voc"

    batch_convert_yolo_to_voc(yolo_dir, img_width, img_height, class_list, output_dir)

Converted Line Detection using Yolo On PalmLeaf\Balinese\Label\1.txt to output_bali_voc\1.xml
Converted Line Detection using Yolo On PalmLeaf\Balinese\Label\10.txt to output_bali_voc\10.xml
Converted Line Detection using Yolo On PalmLeaf\Balinese\Label\11.txt to output_bali_voc\11.xml
Converted Line Detection using Yolo On PalmLeaf\Balinese\Label\12.txt to output_bali_voc\12.xml
Converted Line Detection using Yolo On PalmLeaf\Balinese\Label\13.txt to output_bali_voc\13.xml
Converted Line Detection using Yolo On PalmLeaf\Balinese\Label\14.txt to output_bali_voc\14.xml
Converted Line Detection using Yolo On PalmLeaf\Balinese\Label\15.txt to output_bali_voc\15.xml
Converted Line Detection using Yolo On PalmLeaf\Balinese\Label\16.txt to output_bali_voc\16.xml
Converted Line Detection using Yolo On PalmLeaf\Balinese\Label\17.txt to output_bali_voc\17.xml
Converted Line Detection using Yolo On PalmLeaf\Balinese\Label\18.txt to output_bali_voc\18.xml
Converted Line Detection using Yolo On Pal

In [17]:
if __name__ == "__main__":
    voc_dir = "output_bali_voc"
    output_csv = "annotations3.csv"
    voc_to_csv(voc_dir, output_csv)

In [18]:
data_ground_truth3 = pd.read_csv('annotations3.csv')
data_ground_truth2

,filename,class,xmin,ymin,xmax,ymax
0,1.jpg,line,19,31,629,145
1,1.jpg,line,16,198,626,312
2,1.jpg,line,13,341,626,443
3,1.jpg,line,10,473,623,575
4,10.jpg,line,14,43,623,161
...,...,...,...,...,...,...
237,8.jpg,line,2,483,630,588
238,9.jpg,line,8,33,631,163
239,9.jpg,line,9,186,634,308
240,9.jpg,line,12,308,635,442


In [19]:
if __name__ == "__main__":
    voc_dir = r"Line Detection using Yolo On PalmLeaf\Balinese\Predicted"
    output_csv = "predicted_annotations3.csv"
    voc_to_csv(voc_dir, output_csv)

In [ ]:
if __name__ == "__main__":
    predicted = pd.read_csv('predicted_annotations3.csv')
    ground_truth = pd.read_csv('annotations3.csv')

    predicted['filename'] = predicted['filename'].str.split('_jpg.rf').str[0] + '.jpg'

    metrics_df = evaluate_metrics(predicted, ground_truth)
    
    print(metrics_df)

    overall_metrics = metrics_df[['IoU', 'HD', 'AvgHD', 'HD95']].mean()
    print("Overall Metrics:")
    print(overall_metrics)

    metrics_df.to_csv('per_file_metrics.csv', index=False)
    overall_metrics.to_csv('overall_metrics.csv', header=True)

100%|██████████| 364/364 [00:00<00:00, 502.46it/s]

    filename       IoU        HD     AvgHD        HD95
0      1.jpg  0.966380  2.828427  2.532248  585.263046
1      1.jpg  0.966380  2.828427  2.532248  585.263046
2      1.jpg  0.966380  2.828427  2.532248  585.263046
3      1.jpg  0.980452  2.236068  1.825141  585.378474
4     10.jpg  0.962107  2.828427  2.178694  514.075088
..       ...       ...       ...       ...         ...
359   93.jpg  0.967230  2.236068  1.825141  418.328989
360   93.jpg  0.962520  2.828427  2.532248  419.297202
361   93.jpg  0.969611  2.828427  2.532248  531.821540
362   95.jpg  0.980743  1.414214  1.414214  424.865768
363   96.jpg  0.973878  2.828427  2.532248  405.003518

[364 rows x 5 columns]
Overall Metrics:
IoU        0.965403
HD         2.672669
AvgHD      2.339087
HD95     532.620185
dtype: float64
